Code to generate dummy outputs for optimization

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import random
from datetime import datetime, timedelta, timezone
import os
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from rasterio.enums import Resampling

class Optimizer():
    def __init__(self):
        #self._entities = [] 
        self._time = self._next_time = self._init_time = None
        self._duration = None
        self._time_step = None           
    
    def execute(self,init_time, duration, time_step):

        self._time = self._next_time = self._init_time = init_time
        self._duration = duration
        self._time_step = time_step 
        
        no_loc = 3 # number of location to be sampled(can be set as random number)

        # Reading the locations from file (all the coordinates withing the mo basin for 1km X 1km resolution is saved in the csv file)
        mo_coords = pd.read_csv('mo_coords.csv',header=None)
        mo_coords= mo_coords.iloc[1:] # excluding the 0 value from header
        mo_coords_np = mo_coords.to_numpy()
        mo_coords_np = mo_coords[0].apply(eval).to_numpy() # to read coordinates as list within array

        # loading template for optimization (template has an xarray dataset with all nan values, it is populated within the loop below, 
        # i.e nan values corresponding to requested locations are changed to 1)

        template_nc = xr.open_dataset('opt_template.nc')

        while self._time < self._init_time + self._duration:
            self._next_time = self._time + self._time_step
            print(f"Current time {self._time}")
            print(f"advancing to {self._next_time}")             
            
            # Read the existing Excel file (throws an error if the file is open)
            req = pd.read_excel('requests.xlsx')
            # Randomly select row indices (3 locations from mo basin)
            sampled_indices = np.random.choice(mo_coords_np.shape[0], no_loc, replace=False)
            # Use the indices to extract the rows
            random_rows = mo_coords_np[sampled_indices]
            # Extract the highest existing ID from the `req` DataFrame (or set to 0 if empty)
            starting_id = req['id'].max() if not req.empty else 0
            # Create a new DataFrame for the random rows with required structure
            new_entries = []

            # Remove timezone info before converting to string
            req_date = self._time.replace(tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
            # assumed that request expires if not fulfilled within 3 days 
            req_exp = (self._time + timedelta(days=3)).replace(tzinfo=None).strftime('%Y-%m-%d %H:%M:%S') 


            # Note to Emmanuel
            # variable 'request' in below loop has your coordinates for visualization, ps check the order of lat and lon
            # the code appends the data to excel, can delete all values before running the code
            # I may add another column to excel sheet indicating the satellite for fulfilling the request   

            for i, request in enumerate(random_rows):
                new_entry = {
                    'id': starting_id + i + 1,            # Assign sequential ID
                    'location': f"{request[1]},{request[0]}",
                    'latitude': request[1],              # Latitude value
                    'longitude': request[0],             # Longitude value
                    'request_date': req_date,            # Request Date
                    'request_expiration': req_exp,       # Computed from Request date
                    'simulation_status': None,           # Placeholder - populated in simulation file
                    'request_status': None,              # Placeholder - populated in simulation file
                    'completion_date': None              # Placeholder - populated in simulation file
                }
                new_entries.append(new_entry)
                # @ Emmanuel - remove this code if not using png file
                template_nc_opt = template_nc
                updated_template = template_nc.sel(x=request[0], y=request[1], method="nearest")
                template_nc_opt.loc[{"x": updated_template.x, "y": updated_template.y}] = 1
                

            # Convert to a DataFrame
            new_rows_df = pd.DataFrame(new_entries)
            # Append the new rows to the existing DataFrame
            req_updated = pd.concat([req, new_rows_df], ignore_index=True)
            # Save the updated DataFrame back to the Excel file
            req_updated.to_excel('requests.xlsx', index=False)

            # Save the netcdf file later converted to png
            template_nc_opt.to_netcdf('optimizer_output.nc')
            # Advancing to next time step
            self._time = self._next_time 
            

In [2]:
s = Optimizer()
from datetime import datetime, timedelta, timezone

start = datetime(2025, 1, 1, tzinfo=timezone.utc)
duration = timedelta(days=5)
time_step = timedelta(days=1)
s.execute(start, duration, time_step)

Current time 2025-01-01 00:00:00+00:00
advancing to 2025-01-02 00:00:00+00:00


C:\Users\dramach6\AppData\Local\Temp\ipykernel_18136\785033434.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  req_updated = pd.concat([req, new_rows_df], ignore_index=True)


PermissionError: [Errno 13] Permission denied: 'c:\\Users\\dramach6\\ASU Dropbox\\Divya Ramachandran\\Everything Research\\Research\\SOS\\Simulation Model\\optimizer_output.nc'